In [1]:
# from google.colab import drive

# # Mount Google Drive to /content/drive
# drive.mount('/content/drive')


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Set the seed for reproducibility
tf.random.set_seed(42)

# Define the path to your dataset
train_data_dir = 'C:/Users/user/OneDrive/Desktop/Sugam Car Detection ML/data2a/training'
validation_data_dir = 'C:/Users/user/OneDrive/Desktop/Sugam Car Detection ML/data2a/validation/'

# Define image size and channels
img_width, img_height = 150, 150
channels = 3  # RGB

# Define the input shape based on your images
input_shape = (img_width, img_height, channels)

# Define the number of classes
num_classes = 3  # front, rear, side

# Define hyperparameters
epochs = 75
batch_size = 32

# Load the VGG16 model with pre-trained weights on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
# Unfreeze some layers for fine-tuning
# Unfreeze some layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = False



# Create a Sequential model and add the VGG16 base model
model = Sequential()
model.add(base_model)

# Flatten the output of the VGG16 base model
model.add(Flatten())

# Add Dense layers with BatchNormalization and Dropout
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())


# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

# Create data generators to augment and normalize the data
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.5,
    zoom_range=0.15,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    brightness_range=[0.2, 1.0],  # Adjust brightness
    channel_shift_range=30.0  # Adjust channel intensity
)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model with early stopping
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping]
)
# Save the model
model.save('car_location_model_vgg.h5')

58889256/58889256 [==============================] - 11s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               3

In [ ]:

# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")

# Make predictions on new images
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))

    # Convert the image to an array
    img_array = image.img_to_array(img)

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Rescale the image to match the training data preprocessing
    img_array /= 255.0

    # Make predictions
    prediction = model.predict(img_array)

    # Get the predicted class index
    predicted_class = np.argmax(prediction)

    # Get class labels from the generator
    classes = list(train_generator.class_indices.keys())

    # Get the class name corresponding to the predicted index
    class_name = classes[predicted_class]

    return class_name, prediction[0]
# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Overall Validation Accuracy: {validation_accuracy * 100:.2f}%")

# Replace 'path/to/new_image.jpg' with the path to your new image
new_image_path = 'C:/Users/user/OneDrive/Desktop/Sugam Car Detection ML/data2a/validation/01-rear/0030.JPEG'
predicted_class, prediction = predict_image(model, new_image_path)

print(f"Predicted Class: {predicted_class}")
print(f"Prediction Probabilities: {prediction}")

# You can also convert the probabilities to percentage
prediction_percentage = [f"{prob * 100:.2f}%" for prob in prediction]
print(f"Prediction Probabilities (Percentage): {prediction_percentage}")


5/5 [==============================] - 32s 6s/step - loss: 0.6749 - accuracy: 0.7125
Validation Accuracy: 71.25%
